In [ ]:
#This project is 

In [160]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import regex as re
from lxml import etree as et
from itertools import repeat
import csv
import random
import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [161]:
header_list = ["Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36 OPR/70.0.3728.178 (Edition Yx 01)",
               "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4255.0 Safari/537.36 Edg/87.0.634.0",
              "Mozilla/5.0 (Linux; arm; Android 10; SM-A605FN) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 YaBrowser/20.8.0.174.00 Mobile Safari/537.36"]

base_url = 'https://www.backloggd.com/games/lib/popular?page='
games_list = []
d = 0

In [162]:
game_links = []

for page_no in tqdm_notebook(range(300)):
    page_url = base_url + str(page_no)
    #print(page_url)
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(page_url, headers = header)
    if webpage.status_code == 200:
        soup1 = BeautifulSoup(webpage.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
        g = soup2.find('div', {'class' : 'row show-release toggle-fade'})
        games = g.find_all('div', {'class' : 'col-2 my-2 px-1 px-md-2'})
        game_ref = [game.find('a').get('href') for game in games]
        game_links.extend(['https://www.backloggd.com' + ref for ref in game_ref])

  0%|          | 0/300 [00:00<?, ?it/s]

In [163]:
print(len(game_links))

10764


In [164]:
cols = ['Title', 'Release Date', 'Team', 'Console', 'Rating', 'Times Listed', 'Numbers of Reviews', 'Genres', 'Summary', 'Plays', 'Playing', 'Backlogs', 'Wishlist']
backloggd = pd.DataFrame(columns = cols)

In [165]:
for link in tqdm_notebook(game_links):
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(link, headers=header)
    if webpage.status_code == 200:
        soup1 = BeautifulSoup(webpage.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
        
        title = soup2.find('div', {'class': 'col-auto pr-1'}).get_text().strip()
        
        release_date = ' '.join(soup2.find('div', {'class': 'col-auto mt-auto pr-0'}).get_text().strip().split()[-3:])
        
        try:
            teams = soup2.find('div', {'class': 'col-auto pl-lg-1 sub-title'})
            teams = teams.find_all('a')
            teams = [i.get_text().strip() for i in teams]
        except:
            teams = np.nan
            
        try:
            consoles = soup2.find('div', {'class': 'col-lg-4 mt-1 mt-lg-2 col-12'})
            consoles = consoles.find_all('a', {'class':'game-page-platform'})
            consoles = [console.get_text().strip() for console in consoles]
        except:
            consoles = np.nan
        
        try:
            rating = float(soup2.find(id='score').get_text().strip()[-3:])
        except:
            rating = np.nan
            
        table = soup2.find_all('div', {'class': 'col-12 mb-1'})
        feats = [f.get_text().strip().split('\n')[0].strip() for f in table]
        results = [r.get_text().strip().split('\n')[-1].strip() for r in table]
        
        nlists = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[0]

        nreviews = soup2.find('div', {'class' : 'col-5 col-xl-auto pl-1'}).get_text().strip().split()[0]
        
        genres = soup2.find_all('p', {'class': 'genre-tag'})
        genres = [genre.get_text().strip() for genre in genres]

        summary = soup2.find(id='collapseSummary').get_text().strip()

        row = [title, release_date, teams, consoles, rating, nlists, nreviews, genres, summary]
        row.extend(results)
        backloggd.loc[len(backloggd.index)] = row 


  0%|          | 0/10764 [00:00<?, ?it/s]

In [166]:
backloggd.head()

,Title,Release Date,Team,Console,Rating,Times Listed,Numbers of Reviews,Genres,Summary,Plays,Playing,Backlogs,Wishlist
0,Elden Ring,"Feb 25, 2022","[FromSoftware, Bandai Namco Entertainment]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",4.5,4.8K,3.2K,"[Adventure, RPG]","Elden Ring is a fantasy, action and open world...",23K,4.3K,6.3K,5.8K
1,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","[Nintendo EPD Production Group No. 3, Nintendo]","[Wii U, Nintendo Switch]",4.4,5.3K,3.3K,"[Adventure, Puzzle]",The Legend of Zelda: Breath of the Wild is the...,38K,3.3K,6K,3.1K
2,Hades,"Dec 07, 2018",[Supergiant Games],"[Windows PC, Mac, PlayStation 4, Xbox One, Pla...",4.3,3.3K,2.3K,"[Adventure, Brawler, Indie, RPG]",A rogue-lite hack and slash dungeon crawler in...,27K,3.8K,8K,4.2K
3,The Legend of Zelda: Tears of the Kingdom,"May 12, 2023","[Nintendo, Nintendo EPD Production Group No. 3]",[Nintendo Switch],4.6,2.3K,1.8K,[Adventure],The Legend of Zelda: Tears of the Kingdom is t...,7K,5.5K,3.7K,5.6K
4,Hollow Knight,"Feb 24, 2017",[Team Cherry],"[Windows PC, Mac, Linux, Nintendo Switch]",4.4,3.6K,2.3K,"[Adventure, Indie, Platform]",A 2D metroidvania with an emphasis on close co...,27K,2.9K,11K,2.8K


In [167]:
backloggd.to_csv('backloggd.csv')